In [ ]:
%pip uninstall -y pandas numpy

Found existing installation: pandas 2.0.3
Uninstalling pandas-2.0.3:
  Successfully uninstalled pandas-2.0.3
Found existing installation: numpy 1.24.4
Uninstalling numpy-1.24.4:
  Successfully uninstalled numpy-1.24.4
Note: you may need to restart the kernel to use updated packages.


You can safely remove it manually.
You can safely remove it manually.


In [ ]:
%pip show pandas
%pip show numpy


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install numpy==1.24.4 pandas==2.0.3

  Using cached numpy-1.24.4-cp311-cp311-win_amd64.whl.metadata (5.6 kB)
  Using cached pandas-2.0.3-cp311-cp311-win_amd64.whl.metadata (18 kB)
Using cached numpy-1.24.4-cp311-cp311-win_amd64.whl (14.8 MB)
Using cached pandas-2.0.3-cp311-cp311-win_amd64.whl (10.6 MB)
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [8]:
#%pip install pandas
%pip install selenium
%pip install selenium webdriver-manager pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [10]:
# Inicializas tu driver y entras a la página
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get("https://censos2017.inei.gob.pe/redatam/")

# Espera implícita para que cargue el frameset
driver.implicitly_wait(10)


In [11]:
def switch_to_index_frame(driver):
    driver.switch_to.default_content()
    driver.switch_to.frame(0)            # frame “Portal”
    driver.switch_to.frame("Index")      # iframe con el sidebar

# 1) Ve al frame del sidebar
switch_to_index_frame(driver)

# 2) Expande ambos niveles de menú
for header_text in ["Lista de Áreas-Mapas Temáticos", "Lista de Preguntas"]:
    hdr = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(
        (By.XPATH, f"//h3[normalize-space()='{header_text}']")
    ))
    # hacemos scroll y clic con JS por si acaso
    driver.execute_script("arguments[0].scrollIntoView(true); arguments[0].click();", hdr)

# 3a) OPCIÓN A: clic directo sobre el enlace por texto de link
link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(
    (By.LINK_TEXT, "Preguntas de Población")
))
driver.execute_script("arguments[0].click();", link)



# 4) Cambia al frame Output para trabajar con el formulario
driver.switch_to.default_content()
driver.switch_to.frame(0)            # frame “Portal”
driver.switch_to.frame("Output")     # frame donde carga el formulario de filtros

print("✅ Llegaste a ‘Preguntas de Población’ y ya estás en el frame de Output")



✅ Llegaste a ‘Preguntas de Población’ y ya estás en el frame de Output


In [12]:
def generador_tabla(driver,filtro):
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait, Select
    from selenium.webdriver.support import expected_conditions as EC

    # asume que ya hiciste:
    # driver.switch_to.default_content()
    # driver.switch_to.frame(0)
    # driver.switch_to.frame("Output")

    wait = WebDriverWait(driver, 10)

    # 1) Espera a que aparezca el <form>
    wait.until(EC.presence_of_element_located((By.NAME, "RpEasyList")))

    # 2) Deseleccionar todas las variables por defecto y seleccionar solo la que quieres
    var_select = Select(driver.find_element(By.NAME, "VARIABLE"))
    var_select.deselect_all()                             # quita Poblacio.PENCARE y Poblacio.C5P41
    var_select.select_by_value("Poblacio.C5P6")           # P: ¿Hace 5 años vivía en este distrito?

    # 3) Nivel de salida → Departamento
    Select(driver.find_element(By.NAME, "OUTPUT")).select_by_value(filtro)

    # 4) (Opcional) Asegúrate de que SELECTION quede en “ALL” para todos los departamentos
    Select(driver.find_element(By.NAME, "SELECTION")).select_by_value("ALL")

    # 5) Ejecutar la consulta
    driver.find_element(By.NAME, "Submit").click()
    return 

In [13]:

def guardar_info(driver):
    # Encuentra la tabla
    tabla = driver.find_element(By.CSS_SELECTOR, "div.CSSTableGenerator table")

    # Extrae filas
    filas = tabla.find_elements(By.TAG_NAME, "tr")

    # Inicializa lista de datos
    data = []

    for fila in filas:
        celdas = fila.find_elements(By.TAG_NAME, "td")
        textos = [celda.text.strip() for celda in celdas if celda.text.strip() != '']

        # Filtramos filas con 5 o más elementos válidos (Código, Nombre, 3 valores)
        if len(textos) >= 5:
            codigo = textos[0]
            nombre = textos[1]
            valores = textos[2:5]

            # Limpia valores
            valores = [v.replace(" ", "").replace(",", "") for v in valores]

            # Verifica que hay 3 valores
            if len(valores) == 3:
                fila_dict = {
                    "Código": codigo,
                    "Departamento": nombre,
                    "Aún no había nacido": valores[0],
                    "Sí, vivía hace 5 años en este distrito": valores[1],
                    "No vivía hace 5 años en este distrito": valores[2]
                }
                data.append(fila_dict)

    # Crea DataFrame
    df = pd.DataFrame(data)

    return df

In [14]:
def volver_a_preguntas_poblacion(driver):
    driver.switch_to.default_content()
    driver.switch_to.frame(0)            # frame “Portal”
    driver.switch_to.frame("Index")      # iframe con el sidebar

    # Expandir "Lista de Preguntas"
    hdr = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(
        (By.XPATH, "//h3[normalize-space()='Lista de Preguntas']")
    ))
    driver.execute_script("arguments[0].scrollIntoView(true); arguments[0].click();", hdr)

    # Clic en "Preguntas de Población"
    link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(
        (By.LINK_TEXT, "Preguntas de Población")
    ))
    driver.execute_script("arguments[0].click();", link)

    # Cambiar a frame Output
    driver.switch_to.default_content()
    driver.switch_to.frame(0)
    driver.switch_to.frame("Output")

    print("🔁 Volviste a ‘Preguntas de Población’")



In [15]:
#Primero filtrado
generador_tabla(driver,"Departam")
df1=guardar_info(driver)
volver_a_preguntas_poblacion(driver)

df1=df1.drop(index=0)

🔁 Volviste a ‘Preguntas de Población’


In [16]:
#Segundo filtrado
generador_tabla(driver,"Provinci")
df2=guardar_info(driver)
volver_a_preguntas_poblacion(driver)

df2=df2.drop(index=0)

🔁 Volviste a ‘Preguntas de Población’


In [17]:
#Tercer filtrado
generador_tabla(driver,"Distrito")
df3=guardar_info(driver)
driver.quit()

df3=df3.drop(index=0)

In [18]:
df1.head()

,Código,Departamento,Aún no había nacido,"Sí, vivía hace 5 años en este distrito",No vivía hace 5 años en este distrito
1,01,Departamento: Amazonas,38682,306105,34597
2,02,Departamento: Áncash,92974,905542,85003
3,03,Departamento: Apurímac,35677,328973,41109
4,04,Departamento: Arequipa,109771,1027473,245486
5,05,Departamento: Ayacucho,54200,489769,72207


In [19]:
df2.head()

,Código,Departamento,Aún no había nacido,"Sí, vivía hace 5 años en este distrito",No vivía hace 5 años en este distrito
1,0101,"Amazonas, provincia: Chachapoyas",4519,41295,9692
2,0102,"Amazonas, provincia: Bagua",7761,61137,5202
3,0103,"Amazonas, provincia: Bongara",2226,20145,3266
4,0104,"Amazonas, provincia: Condorcanqui",6693,34708,1069
5,0105,"Amazonas, provincia: Luya",4325,36941,3170


In [20]:
df3.head()

,Código,Departamento,Aún no había nacido,"Sí, vivía hace 5 años en este distrito",No vivía hace 5 años en este distrito
1,010101,"Amazonas, Chachapoyas, distrito: Chachapoyas",2633,22774,7182
2,010102,"Amazonas, Chachapoyas, distrito: Asunción",22,220,20
3,010103,"Amazonas, Chachapoyas, distrito: Balsas",111,938,87
4,010104,"Amazonas, Chachapoyas, distrito: Cheto",44,533,65
5,010105,"Amazonas, Chachapoyas, distrito: Chiliquin",49,483,53


In [21]:
df1.info(
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 1 to 25
Data columns (total 5 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   Código                                  25 non-null     object
 1   Departamento                            25 non-null     object
 2   Aún no había nacido                     25 non-null     object
 3   Sí, vivía hace 5 años en este distrito  25 non-null     object
 4   No vivía hace 5 años en este distrito   25 non-null     object
dtypes: object(5)
memory usage: 1.1+ KB


In [22]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 1 to 196
Data columns (total 5 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   Código                                  196 non-null    object
 1   Departamento                            196 non-null    object
 2   Aún no había nacido                     196 non-null    object
 3   Sí, vivía hace 5 años en este distrito  196 non-null    object
 4   No vivía hace 5 años en este distrito   196 non-null    object
dtypes: object(5)
memory usage: 7.8+ KB


In [23]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1874 entries, 1 to 1874
Data columns (total 5 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   Código                                  1874 non-null   object
 1   Departamento                            1874 non-null   object
 2   Aún no había nacido                     1874 non-null   object
 3   Sí, vivía hace 5 años en este distrito  1874 non-null   object
 4   No vivía hace 5 años en este distrito   1874 non-null   object
dtypes: object(5)
memory usage: 73.3+ KB


In [24]:
df1.to_csv("Movimiento_Departamento.csv", index=False)

In [25]:
df2.to_csv("Movimiento_Provincia.csv", index=False)

In [26]:
df3.to_csv("Movimientos_Distrito.csv", index=False)